```
pip install opentelemetry-api
pip install opentelemetry-sdk
pip install opentelemetry-exporter-otlp
```

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()


In [ ]:
import logging
from opentelemetry.exporter.otlp.proto.http._log_exporter import OTLPLogExporter
from opentelemetry._logs import set_logger_provider
from opentelemetry.sdk._logs import LoggerProvider, LoggingHandler
from opentelemetry.sdk._logs.export import BatchLogRecordProcessor
from opentelemetry.sdk.resources import SERVICE_NAME, Resource

seqEndpoint = os.environ["SEQ_INGEST_ENDPOINT"]
serviceName = "langchain"

# Service name is required for most backends
resource = Resource(attributes={SERVICE_NAME: serviceName})

# configure logging
logger_provider = LoggerProvider(resource=resource)
logger_provider.add_log_record_processor(
    BatchLogRecordProcessor(OTLPLogExporter(endpoint=seqEndpoint))
)
set_logger_provider(logger_provider)
handler = LoggingHandler(logger_provider=logger_provider)

logging.basicConfig()
logging.getLogger().addHandler(handler)

In [ ]:
# logging.basicConfig()
# logging.getLogger("MINHAZ").setLevel(logging.DEBUG)
logger = logging.getLogger("MINHAZ")
logger.setLevel(logging.DEBUG)
# logger.warning("Testing warning 2")
# logger.debug("Testing debug")
logger.info("Testing info 2", extra={"lat": -26, "lon": 152})


def cleanup():
    logger_provider.shutdown()

In [ ]:
from uuid import UUID
from langchain_core.callbacks import BaseCallbackHandler
from typing import Any, Dict, List, Optional, Sequence, Union
from langchain_core.callbacks import BaseCallbackHandler
from langchain_core.messages import BaseMessage
from langchain_core.outputs import LLMResult
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.outputs.generation import GenerationChunk
from langchain_core.outputs.chat_generation import ChatGenerationChunk
from langchain_core.documents.base import Document
from tenacity import RetryCallState
import logging


class LoggingHandler(BaseCallbackHandler):
    def __init__(self):
        self.logger = logging.getLogger("LoggingHandler")
        self.logger.setLevel(logging.DEBUG)

    def on_agent_action(
        self,
        action: AgentAction,
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        **kwargs: Any,
    ) -> None:
        print("Agent Action")

    def on_agent_finish(
        self,
        finish: AgentFinish,
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        **kwargs: Any,
    ) -> None:
        print("Agent finish")

    def on_chain_end(
        self,
        outputs: Dict[str, Any],
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        **kwargs: Any,
    ) -> None:
        print("Chain end")

    def on_chain_error(
        self,
        error: BaseException,
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        **kwargs: Any,
    ) -> None:
        print("Chain Error")

    def on_chain_start(
        self,
        serialized: Dict[str, Any],
        inputs: Dict[str, Any],
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        tags: Optional[List[str]] = None,
        metadata: Optional[Dict[str, Any]] = None,
        **kwargs: Any,
    ) -> None:
        print("Chain Start")

    def on_chat_model_start(
        self,
        serialized: Dict[str, Any],
        messages: List[List[BaseMessage]],
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        tags: Optional[List[str]] = None,
        metadata: Optional[Dict[str, Any]] = None,
        **kwargs: Any,
    ) -> None:
        print("on_chat_model_start")

    def on_llm_error(
        self,
        error: BaseException,
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        **kwargs: Any,
    ) -> None:
        print("on_llm_error")

    def on_llm_new_token(
        self,
        token: str,
        *,
        chunk: Optional[Union[GenerationChunk, ChatGenerationChunk]] = None,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        **kwargs: Any,
    ) -> None:
        print("on_llm_new_token")

    def on_llm_start(
        self,
        serialized: Dict[str, Any],
        prompts: List[str],
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        tags: Optional[List[str]] = None,
        metadata: Optional[Dict[str, Any]] = None,
        **kwargs: Any,
    ) -> None:
        self.logger.info(
            "LLM Start %s",
            prompts.__str__(),
            extra={
                "serialized": serialized.__str__(),
                "prompts": prompts.__str__(),
                "run_id": run_id.__str__(),
                "parent_run_id": parent_run_id.__str__(),
                "tags": tags.__str__(),
                "metadata": metadata.__str__(),
            },
        )

    def on_llm_end(
        self,
        response: LLMResult,
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        **kwargs: Any,
    ) -> None:
        self.logger.info(
            "LLM END %s",
            response.__str__(),
            extra={
                "response": response.__str__(),
                "run_id": run_id.__str__(),
                "parent_run_id": parent_run_id.__str__(),
            },
        )

    def on_retriever_end(
        self,
        documents: Sequence[Document],
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        **kwargs: Any,
    ) -> None:
        print("on_retriever_end")

    def on_retriever_error(
        self,
        error: BaseException,
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        **kwargs: Any,
    ) -> None:
        print("on_retriever_error")

    def on_retriever_start(
        self,
        serialized: Dict[str, Any],
        query: str,
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        tags: Optional[List[str]] = None,
        metadata: Optional[Dict[str, Any]] = None,
        **kwargs: Any,
    ) -> None:
        print("on_retriever_start")

    def on_retry(
        self,
        retry_state: RetryCallState,
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        **kwargs: Any,
    ) -> None:
        print("on_retry")

    def on_text(
        self,
        text: str,
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        **kwargs: Any,
    ) -> None:
        print("on_text")

    def on_tool_end(
        self,
        output: Any,
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        **kwargs: Any,
    ) -> None:
        print("on_tool_end")

    def on_tool_error(
        self,
        error: BaseException,
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        **kwargs: Any,
    ) -> None:
        print("on_tool_error")

    def on_tool_start(
        self,
        serialized: Dict[str, Any],
        input_str: str,
        *,
        run_id: UUID,
        parent_run_id: Optional[UUID] = None,
        tags: Optional[List[str]] = None,
        metadata: Optional[Dict[str, Any]] = None,
        inputs: Optional[Dict[str, Any]] = None,
        **kwargs: Any,
    ) -> None:
        print("on_tool_start")

    # def on_llm_new_token(self, token: str, **kwargs) -> None:
    #     print(f"LLM New Token, token: {token}")

    # def on_llm_start(self, serialized, prompts, **kwargs) -> None:
    #     print(f"LLM start: {prompts}")

    # def on_chat_model_start(
    #     self, serialized: Dict[str, Any], messages: List[List[BaseMessage]], **kwargs
    # ) -> None:
    #     print("Chat model started")

    # def on_llm_end(self, response: LLMResult, **kwargs) -> None:
    #     print(f"Chat model ended, response: {response}")

    # def on_chain_start(
    #     self, serialized: Dict[str, Any], inputs: Dict[str, Any], **kwargs
    # ) -> None:
    #     print(f"Chain {serialized.get('name')} started")

    # def on_chain_end(self, outputs: Dict[str, Any], **kwargs) -> None:
    #     print(f"Chain ended, outputs: {outputs}")


loggingHandler = LoggingHandler()
callbacks = [loggingHandler]

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint

repo_id = "mistralai/Mistral-7B-Instruct-v0.2"
huggingFaceApiToken = os.environ["HUGGINGFACEHUB_API_TOKEN"]
llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    temperature=0.5,
    huggingfacehub_api_token=huggingFaceApiToken,
    callbacks=callbacks,
)

In [ ]:
llm.invoke("Give me an animal's name that is white.", config={"callbacks": callbacks})

In [ ]:
# cleanup()